In [14]:
import torch
import torchvision
import csv

import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.utils as vutils
import torchvision.transforms as transforms
import random

import pandas as pd
import matplotlib.pyplot as plt

In [29]:
# set the device to run the model on
device = 'cuda'
# total training steps
num_steps = 50000
# training batch size
batch_size = 8000
# how fast the learning rate is 
learn_rate = 0.005
# technique used in gradient descent to overcome local minima and oscilation
momentum = 0.9
# colour channels, 4 if alpha is present
num_channels = 3
# dimensions of input image
image_shape = (512, 644)

In [16]:
class CPPN(nn.Module):
    # defines initilizatoin class
    def __init__(self):
      # initiates itself
      super(CPPN, self).__init__()
      # fully connected layer 1, with 2 inputs and 16 outputs
      self.fc1 = nn.Linear(2, 64)
      # layer 2, 16 inputs, 32 outputs
      self.fc2 = nn.Linear(64, 64)
      self.fce1 = nn.Linear(64, 64)
      # layer 3, 32 inputs, output dependent on the number of colour channels
      self.fc3 = nn.Linear(64, num_channels)     
    
    # defines forward pass
    def forward(self, x):
        # forward through first layer
        x = self.fc1(x)
        # rectified linear activation function 
        x = F.relu(x)
        # layer 2
        x = self.fc2(x)
        # another reclinear function
        x = F.relu(x)
        x = self.fce1(x)
        x = F.relu(x)
        # layer 3
        x = self.fc3(x)
        # sigmoid activation function
        x = F.sigmoid(x)
        # returns the output of layer 3 after activation
        return x

In [17]:
cppn = CPPN()
cppn.laod_state_dict = torch.load('CPPN_model.pt')
print(cppn)
# feature maps trasfer learning 

CPPN(
  (fc1): Linear(in_features=2, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fce1): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=3, bias=True)
)


In [18]:
# index, test = enumerate(cppn.named_parameters())
# print(test)

In [19]:
parameter_info = []
fc1 = []
full_coords = []
full_weights = []

def spatial_coords(array, layer):

    coords = []
    row1 = []
    row2 = []
    for i in range(array.shape[0]):
        row2.append([layer+1, i])
    for i in range(array.shape[1]):
        row1.append([layer, i])
            
    array = array.flatten("C").tolist()
    # print(array)
    for i in row2:
        for j in row1:
            temp = []
            temp.extend(j)
            temp.extend(i)
            coords.append((temp))
    coords_array = np.array(coords)
    # coords_array.astype(int)
    # output = np.column_stack((coords_array, array))

    # print(len(output))
    return coords, array
            
    # array.flatten()
    # return np.stack((array, coords))

index = 0
for name, param in cppn.named_parameters():
    
    # print(name)
    if name.endswith(".weight"):
        
        # print(index)
        print(param)
        temp_layer = param.detach().numpy() # need to learn more about gradients and why they are required
        # print(temp_layer)

        temp_coords, temp_weights = spatial_coords(temp_layer, index)
        full_coords.extend(temp_coords)
        full_weights.extend(temp_weights)

        index += 1
        # print(fc1.shape[0])


print(full_coords)
print(full_weights)
    
# layer1_coords, layer1_weights = spatial_coords(fce1, 2)

# print(layer1_arragned)
    
        

Parameter containing:
tensor([[-0.5907,  0.2571],
        [ 0.4231,  0.4184],
        [ 0.6556, -0.0220],
        [-0.3731,  0.6824],
        [-0.6706,  0.1860],
        [ 0.5126,  0.6431],
        [-0.5465,  0.4412],
        [ 0.0497,  0.6256],
        [-0.1493,  0.4146],
        [-0.3134, -0.6466],
        [ 0.2412,  0.6142],
        [-0.2741, -0.1626],
        [-0.3792,  0.1930],
        [-0.0843,  0.0808],
        [ 0.3651,  0.4468],
        [ 0.5930,  0.0495],
        [ 0.4064,  0.6089],
        [ 0.1798,  0.2957],
        [ 0.6836,  0.0508],
        [-0.5053,  0.6489],
        [-0.6876, -0.4635],
        [-0.5434, -0.1784],
        [ 0.1412, -0.5597],
        [ 0.6967,  0.0217],
        [ 0.6130,  0.3412],
        [ 0.5298, -0.1192],
        [-0.4247,  0.0698],
        [-0.6501,  0.6780],
        [-0.1558,  0.6190],
        [-0.4153,  0.4842],
        [ 0.4757,  0.5950],
        [ 0.0387, -0.5985],
        [-0.0072,  0.2669],
        [-0.3895, -0.3010],
        [ 0.5464,  0.0967]

In [20]:
class CPPN_squared(nn.Module):
    # defines initilizatoin class
    def __init__(self):
      # initiates itself
      super(CPPN_squared, self).__init__()
      # fully connected layer 1, with 2 inputs and 16 outputs
      self.fc1 = nn.Linear(4, 16)
      # layer 2, 16 inputs, 32 outputs
      # self.fc2 = nn.Linear(16, 32)
      # self.fce1 = nn.Linear(32, 64)
      # self.fce2 = nn.Linear(64,128)
      # layer 3, 32 inputs, output dependent on the number of colour channels
      self.fc3 = nn.Linear(16, 1)     
    
    # defines forward pass
    def forward(self, x):
        # forward through first layer
        x = self.fc1(x)
        # rectified linear activation function 
        x = F.relu(x)
        # layer 2
        # x = self.fc2(x)
        # # another reclinear function
        # x = F.relu(x)
        # x = self.fce1(x)
        # x = F.relu(x)
        # x = self.fce2(x)
        # x = F.relu(x)
        # # layer 3
        x = self.fc3(x)
        # sigmoid activation function
        x = F.sigmoid(x)

        # returns the output of layer 3 after activation
        return x

In [21]:
CPPN_squared = CPPN_squared()
CPPN_squared.to(device)
CPPN_squared.requires_grad_()

# optimiser = torch.optim.SGD(cppn.parameters(), lr=learn_rate, momentum=momentum)
optimiser = torch.optim.Adam(CPPN_squared.parameters(), lr=learn_rate)

criterion = nn.MSELoss(reduction='sum')

# criterion = nn.CrossEntropyLoss(reduction='sum')

In [22]:
def normalize_minus1_to_1(tensor):
    min_vals, _ = torch.min(tensor, dim=0)
    max_vals, _ = torch.max(tensor, dim=0)
    normalized_tensor = 2 * (tensor - min_vals) / (max_vals - min_vals) - 1
    return normalized_tensor

In [23]:
# print(len(layer1_weights))
all_xy_coordinates = torch.tensor(full_coords, device=device, dtype=torch.float32)
print(all_xy_coordinates)

# mean_coords = all_xy_coordinates.mean(dim=0)
# std_coords = all_xy_coordinates.std(dim=0)

# Normalize the coordinates matrix along each column
normalized_coords_matrix = normalize_minus1_to_1(all_xy_coordinates)


all_pixel_values = torch.tensor(full_weights, device=device, dtype=torch.float32)
all_pixel_values = (all_pixel_values + 1)/2
# all_pixel_values = torch.unsqueeze(all_pixel_values, 0)
print(all_pixel_values)

tensor([[ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  1.,  0.],
        [ 0.,  0.,  1.,  1.],
        ...,
        [ 3., 61.,  4.,  2.],
        [ 3., 62.,  4.,  2.],
        [ 3., 63.,  4.,  2.]], device='cuda:0')
tensor([0.2047, 0.6285, 0.7116,  ..., 0.4919, 0.5098, 0.5363], device='cuda:0')


In [24]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(normalized_coords_matrix, test_size= 0.2, random_state=42)

In [25]:
print(normalized_coords_matrix)

tensor([[-1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000, -0.9683, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000, -0.9683],
        ...,
        [ 1.0000,  0.9365,  1.0000, -0.9365],
        [ 1.0000,  0.9683,  1.0000, -0.9365],
        [ 1.0000,  1.0000,  1.0000, -0.9365]], device='cuda:0')


In [30]:
num_coords = all_xy_coordinates.shape[0]
coord_indexes = list(range(0, num_coords))
losses = []
img_list = []
running_loss = 0.0
best_loss = 100000

for i in range(num_steps):
    optimiser.zero_grad()
    cppn.zero_grad()

    # Sample a random batch of indexes from the list coord_indexes
    batch_indexes = torch.tensor(np.array(random.sample(coord_indexes, batch_size)))
    
    # Get batch of respective xy_coordiantes
    coordinates_batch = normalized_coords_matrix[batch_indexes]
    
    # And respective pixel values 
    pixel_values_batch = all_pixel_values[batch_indexes]
    pixel_values_batch = pixel_values_batch.unsqueeze(1)
    
    # Process data with model
    approx_pixel_values = CPPN_squared(coordinates_batch)
    
    # Calculate and track loss function
    loss = criterion(pixel_values_batch, approx_pixel_values)
    running_loss += loss.item()
    losses.append(loss.item())
    
    if i % 100 == 0:
        if running_loss < best_loss:
            best_loss = running_loss
            torch.save(CPPN_squared.state_dict(), 'CPPN2model.pt')
        print(f'step {i}, loss {running_loss/1000:.3f}')
        running_loss = 0.0
        with torch.no_grad():
            prediction = CPPN_squared(normalized_coords_matrix)
            print(prediction)
            # prediction  = torch.swapaxes(prediction, 0, 1)
            # prediction = torch.reshape(prediction, (num_channels, image_shape[1], image_shape[0]))
            # if not os.path.exists('training_ims'):
            #     os.makedirs('training_ims')
            # save_image(prediction, f'training_ims/im_{int(i/1000):06}.png')
            # img_list.append(prediction)

            
    #Update model
    loss.backward()
    optimiser.step()

step 0, loss 0.015
tensor([[0.5664],
        [0.5665],
        [0.5646],
        ...,
        [0.4997],
        [0.4987],
        [0.4976]], device='cuda:0')
step 100, loss 1.485
tensor([[0.5741],
        [0.5741],
        [0.5723],
        ...,
        [0.4968],
        [0.4957],
        [0.4946]], device='cuda:0')
step 200, loss 1.483
tensor([[0.5746],
        [0.5746],
        [0.5727],
        ...,
        [0.4997],
        [0.4986],
        [0.4975]], device='cuda:0')
step 300, loss 1.484
tensor([[0.5780],
        [0.5780],
        [0.5761],
        ...,
        [0.4964],
        [0.4952],
        [0.4940]], device='cuda:0')
step 400, loss 1.484
tensor([[0.5767],
        [0.5766],
        [0.5748],
        ...,
        [0.4980],
        [0.4969],
        [0.4958]], device='cuda:0')
step 500, loss 1.485
tensor([[0.5746],
        [0.5745],
        [0.5727],
        ...,
        [0.4995],
        [0.4984],
        [0.4974]], device='cuda:0')
step 600, loss 1.483
tensor([[0.5709],
   